# Unmasking Hotel Identities

In which we try to match hotel IDs to the actual hotel in NYC, given distances from four different attractions.

In [16]:
# imports...
import pandas, numpy as np, warnings

# importing helper methods
from util import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# suppressing all warnings (probably bad practice!)
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mapping Hotel IDs to Addresses or Name of Hotel

Using the (ID, distances) workbook curated by our data providers and the (name, address, distances) workbook we created in the "Getting Hotel Distances From Attractions" Jupyter notebook, we match up each hotel ID with a (name, address) pair. We choose the "match" to be the two entries which minimize the absolute difference, in miles, between the distance values for the two entries.

In [24]:
print '... loading PropertyInfoFile workbook'

# load up the (ID, distances) worksheet
id_dist = pandas.read_excel('../data/PropertyInfoFile-to-Dan.xlsx')

print '... loading hotel names, addresses, and distances workbook'

# load up the (name, address, distances) worksheet
name_addr_dist = pandas.read_excel('../data/Hotel_Names_Addresses_Distances.xlsx')

# create new dataframe for (name, address, ID) entries
sLength = len(name_addr_dist['Name'])
name_addr_ID = name_addr_dist.copy()

# drop useless columns
name_addr_ID = name_addr_ID.drop('Broadway Theatre Distance', axis=1)
name_addr_ID = name_addr_ID.drop('Metropolitan Museum of Art Distance', axis=1)
name_addr_ID = name_addr_ID.drop('Empire State Building Distance', axis=1)
name_addr_ID = name_addr_ID.drop('One World Trade Center Distance', axis=1)

# create new column for ID matching
name_addr_ID.loc[:, 'Share ID'] = pandas.Series(np.zeros(sLength), index=name_addr_dist.index)

print '... finding best distance matches per hotel\n'

# keep track of rows we have already used in new sheet
used_rows = []

# loop through every row in the (ID, distances) worksheet
for idx, id_dist_row in id_dist.iterrows():
    # print progress to console
    if idx % 10 == 0:
        print 'Progress:', idx, '/', len(id_dist['Share ID'])
    
    # keep track of the best matching row
    best_row, best_dist = None, np.inf
    # loop through every row in the (name, address, distances) worksheet
    for row_idx, name_addr_dist_row in name_addr_dist.iterrows():
            
        # calculate absolute value distances
        dist1 = abs(id_dist_row['Broadway Theatre'] - name_addr_dist_row['Broadway Theatre Distance'])
        dist2 = abs(id_dist_row['Metropolitan Museum of Art'] - name_addr_dist_row['Metropolitan Museum of Art Distance'])
        dist3 = abs(id_dist_row['Empire State Building'] - name_addr_dist_row['Empire State Building Distance'])
        dist4 = abs(id_dist_row['One World Trade Center'] - name_addr_dist_row['One World Trade Center Distance'])
        
        # combine them into total absolute value distance
        dist = dist1 + dist2 + dist3 + dist4
        
        # if this is the closest distance thus far, store it and the row as the best
        if dist < best_dist and row_idx not in used_rows:
            best_dist = dist
            best_row = name_addr_dist_row
            best_row_idx = row_idx
        else:
            continue
    
    # create entry in new dataframe for the closest match
    name_addr_ID['Share ID'][idx] = id_dist_row['Share ID']
    name_addr_ID['Name'][idx], name_addr_ID['Address'][idx] = best_row['Name'], best_row['Address']
    
    # add row to used row list
    used_rows.append(best_row_idx)
        
print 'Progress:', len(id_dist['Share ID']), '/', len(id_dist['Share ID'])

# save out new (name, address, ID) spreadsheet
writer = pandas.ExcelWriter('../data/Name_Address_ID.xlsx')
name_addr_ID.to_excel(writer, 'Names, Addresses, IDs')
writer.close()

... loading PropertyInfoFile workbook
... loading hotel names, addresses, and distances workbook
... finding best distance matches per hotel

Progress: 0 / 178
Progress: 10 / 178
Progress: 20 / 178
Progress: 30 / 178
Progress: 40 / 178
Progress: 50 / 178
Progress: 60 / 178
Progress: 70 / 178
Progress: 80 / 178
Progress: 90 / 178
Progress: 100 / 178
Progress: 110 / 178
Progress: 120 / 178
Progress: 130 / 178
Progress: 140 / 178
Progress: 150 / 178
Progress: 160 / 178
Progress: 170 / 178
Progress: 178 / 178


## Viewing the Data

Let's take a look!

In [25]:
name_addr_ID

,Name,Address,Share ID
0,Homewood Suites New York Midtown Manhattan Tim...,312 W 37th St New York NY 10018-4208,80307.0
1,Hilton New York Fashion District,152 W 26th St New York NY 10001-6801,81620.0
2,Holiday Inn New York City Times Square,585 8th Ave New York NY 10018-3003,82671.0
3,Doubletree New York Times Square South,341 W 36th St New York NY 10018-6401,83645.0
4,Hampton Inn ManhattanTimes Square South,337 W 39th St New York NY 10018-1401,83706.0
5,The Ludlow Hotel,180 Ludlow St New York NY 10002-1514,84151.0
6,Hyatt Place New York Midtown South,52 W 36th St New York NY 10018-8029,84346.0
7,Marriott New York Downtown,85 West St New York NY 10006-1532,84511.0
8,Hyatt Times Square,135 W 45th St New York NY 10036-4004,84541.0
9,Courtyard New York Manhattan Herald Square,71 W 35th St New York NY 10001-2112,85007.0


In [26]:
for idx, val in enumerate(name_addr_ID.duplicated('Name')):
    if val:
        print idx